# Retrieve Twitter Data Programatically and save to CSV #

I used twarc library to retrieve data, and save to csv. The library is easy to use, and sample code works.

**This notebook is a proof of concept, only to be used for testing**

## Setup ##
* Download Notebook from github
* Install libary (twarc) using pip command, preferred installer program for python language
* Ensure the data-output folder is setup, the query csv results will get stored in this folder

## Run get twitter api in notebook ##
* Update bearer_token with your value saved in a safe place. **Dont share your bearer_token**
* Enter query value, e.g. **"Tesla"**
* Update start_time and end_time values. If testing with Tesla, use **minutes**. If testing the query on your brand switch minutes to **hours**

In [ ]:
import json
from datetime import datetime, timezone, timedelta

from twarc.client2 import Twarc2
from twarc_csv import CSVConverter

# Your bearer token here
t = Twarc2(bearer_token="XXXX")

# Start and end times must be in UTC
start_time = datetime.now(timezone.utc) + timedelta(minutes=-2)
# end_time cannot be immediately now, has to be at least 30 seconds ago.
end_time = datetime.now(timezone.utc) + timedelta(minutes=-1)

#query = "dogs lang:en -is:retweet has:media"
query = "Tesla"

print(f"Searching for \"{query}\" tweets from {start_time} to {end_time}...")

# search_results is a generator, max_results is max tweets per page, not total, 100 is max when using all expansions.
search_results = t.search_recent(
    query=query, start_time=start_time, end_time=end_time, max_results=30)

# Get all results page by page:
for page in search_results:
    # Do something with the page of results:
    with open("data-output/brand_results.jsonl", "w+") as f:
        f.write(json.dumps(page) + "\n")
    print("Wrote a page of results...")

print("Converting to CSV...")

# This assumes `results.jsonl` is finished writing.
with open("data-output/brand_results.jsonl", "r") as infile:
    with open("data-output/brand_output.csv", "w") as outfile:
        converter = CSVConverter(infile, outfile)
        converter.process()

print("Finished.")



## Run Sentiment analysis on the tweets ##

### Option 1 create a dataframe to loop through the tweet data, and call sentiment analysis API in AWS ###
* Import pandas library. If you dont' have pandas run pip3 install pandas (from a cell in the notebook) 
* Import boto3, which is AWS SDK
* Change filename to match your file location (where your CSV file is stored)
* Add or substract columns names, i picked created_at, text, source, and authorname
* Review the results
* **This is a proof of concept to demonstrate how quickly you can determine sentiment on text in your data set** There are optimal ways to set this up in a production scenario

In [ ]:
import pandas as pd
import boto3
comprehend = boto3.client('comprehend')
filemame = './XXX' # dont forget .csv
df = pd.read_csv(filemame)

for index, row in df.iterrows():
    #print(row)
    print(row['created_at'], row['text'], row['source'],row['author.name'],comprehend.detect_sentiment(Text=row['text'],LanguageCode='en')['Sentiment'])

### Option 2 use CSV library in python to loop through the tweet data, and call sentiment analysis API in AWS ###
* change filename to match your file location (where your CSV file is stored)
* determine column location (in my example, column 14 = text). Sentiment analysis requires text.
* review the results

In [ ]:
import csv
filename = './XXXX' # dont forget .csv
with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        #print(row)
        print(row[14], row[15], row[17],comprehend.detect_sentiment(Text=row[14],LanguageCode='en')['Sentiment'])